<!-- dom:TITLE: Programmation Python  pour les mathématiques -->
# Python Programming for Mathematics
<!-- dom:AUTHOR: Julien Guillod at [Sorbonne Université](http://www.sorbonne-universite.fr/), -->
<!-- Author: -->  
**Julien Guillod**, [Sorbonne Université](http://www.sorbonne-universite.fr/),
Licensed <a href="https://creativecommons.org/licenses/by-nc-nd/4.0/">CC BY-NC-ND</a>


All chapters are available in
[HTML](https://python.guillod.org//) and [PDF](https://python.guillod.org//python.pdf).
This notebook is also executed on [mybinder](https://mybinder.org/v2/gh/juguillod/python/master?filepath=chap05.ipynb).






# 5 Graph theory
<div id="ch:graphes"></div>

A graph is a pair $G = (X, E)$ made of a non-empty and finite set $X$, and another set $E$ of pairs of elements in $X$. The elements in $X$ represent the vertices of the graph $G$, and those in $E$ are edges of $G$.
A graph is oriented when there are directions, meaning the pairs of elements in $E$ are ordered lists, meaning $(i,j) \in E$ is not equivalent to $(j,i)\in E$. The scope of this lesson considers only non-oriented graphs, meaning the order of any pair of elements of $X$ are not taken into account ($\{i,j\} \in E$).

For example, a complete graph of $n$ vertices $K_n$ is a graph with $X = {1,2,\dots,n}$ having as edges all possible combinations of pair of elements in $X$. In particular, $K_4 = (X,E)$ where $X=\{1,2,3,4\}$ and $E = \big\{ \{1,2\}, \{1,3\}, \{1,4\}, \{2, 3\}, \{2, 4\}, \{3, 4\}  \big\}$.

**This chapter covers:**

* non-oriented graphs

* dictionary-like representation

* utilisation of frozensets

* matrix of adjacency

* finding path and triangles

* recursive functions





<!-- --- begin exercise --- -->

# Exercise 5.1: Dictionary-like graph

One way to represent $G$ is to create a Python dictionary where the keys represent the graph's vertices, and the associated values $x \in X$ represents its neighbors.


**a)**
Construct the following graphs using dictionaries:
<!-- dom:FIGURE: [https://python.guillod.org/fig/graphes.png, width=800 frac=0.9] -->
<!-- begin figure -->

<p></p>
<img src="https://python.guillod.org/fig/graphes.png" width=800>

<!-- end figure -->



In [1]:
g1 = {1:{2,5}, 2:{1,3,5}, 3:{2,4}, 4:{3,5,6}, 5:{1,2,4}, 6:{4}}
g2 = {1:{2,3,4,5}, 2:{1,3,4,5}, 3:{1,2,4,5}, 4:{1,2,3,5}, 5:{1,2,3,4}}
g3 = {1:{3,4,6}, 2:{4,5,7}, 3:{1,5,8}, 4:{1,2,9}, 5:{2,3,10}, 6:{1,7,10}, 7:{2,6,8}, 8:{3,7,9}, 9:{4,8,10}, 10:{5,6,9}}


**b)**
Write a function `complete(n)` constructing a dictionary of a complete graph $K_n$.


In [2]:
def complete(n):
    g = dict()
    vertices = {s for s in range(1,n+1)}
    for i in range(1, n+1):
        g[i] = vertices - {i}
    return g

complete(6)

{1: {2, 3, 4, 5, 6},
 2: {1, 3, 4, 5, 6},
 3: {1, 2, 4, 5, 6},
 4: {1, 2, 3, 5, 6},
 5: {1, 2, 3, 4, 6},
 6: {1, 2, 3, 4, 5}}


**c)**
A graph's dictionary contains one information multiple times. Write a function `correction(graph)` correcting any missing elements so that for any vertex `x`, if `y` is in `graph[x]` then `y` must also be a key and `x` must be in `graph[y]`. Test the function afterwards.


In [3]:
def correction(g):
    corrected = dict()
    
    # First we loop through each key-element pair in the g dictionary, if the key is not already in the final results, we add
    # this key as a new key. Then we loop through its set of values, which contains all of its neighbors. If the neighbor does
    # not yet exist as key in corrected, we add a new set as value with first element as the current neighbor's dictionary key.
    # If existed, we add this key to the current set. Sounds complex but actually very easy to understand if you look at the code.
    # In the end we must make sure that for any value in the set of each key, the same key must exist in the set that that
    # value holds as key.
    
    for vertex in g:
        if vertex not in corrected:
            corrected[vertex] = set()
        for neighbor in g[vertex]:
            if neighbor not in corrected:
                corrected[neighbor] = {vertex}
            else:
                corrected[neighbor].add(vertex)
            corrected[vertex].add(neighbor)
    return corrected

g = {1:{2,3,5},5:{2,4},3:{1,5}}
correction(g)

{1: {2, 3, 5}, 2: {1, 5}, 3: {1, 5}, 5: {1, 2, 3, 4}, 4: {5}}


**d)**
Write a function returning the set (type `set`) of all edges of a graph from its dictionary.

In [4]:
# By convention, we should write the edges as a pair (i.e. tuple) of vertices with the smaller valued vertex first

def edges(g):
    all_edges = set()
    for x in g:
        for y in g[x]:
            if (min(x,y),max(x,y)) not in all_edges:
                all_edges.add((min(x,y),max(x,y)))
    return all_edges

g = correction(g)
edges(g)

{(1, 2), (1, 3), (1, 5), (2, 5), (3, 5), (4, 5)}


**e)**
<span style="color:red">!</span> Write a function returning whether 2 vertices are connected or not, and if so return the path between these 2 vertices.

<!-- --- begin hint in exercise --- -->

**Indication:**
Write a recursive function.

<!-- --- end hint in exercise --- -->



In [5]:
def path_finder(g, a, b, passage = []):
    # This recursive function uses the DFS (depth first search) algorithm looping through all neighbors of vertex a to
    # look for b. A list of passage is memorised to keep track of visited vertices. If we cannot find b in the
    # neighborhood of a, we continue to look into all neighbors of those of a, until we reach the end of its
    # connectivity (all points that connected together). In case of not finding b in a's connectivity, return None
    
    passage.append(a)
    if b in g[a]:
        passage.append(b)
        return passage
    for neigh in g[a]:
        if neigh not in passage:
            return path_finder(g, neigh, b, passage)
    return None    
        
g = {1:{2,5}, 2:{1,3,5}, 3:{2,4}, 4:{3,5,6}, 5:{1,2,4}, 6:{4}, 7:{8,9}, 8:{7,9}, 9:{7,8}}
print(path_finder(g, 1, 4))
print(path_finder(g, 2, 8))

[1, 2, 3, 4]
None


**f)**
<span style="color:red">!</span> Write a function returning all possible paths between 2 vertices (without cycle).

In [ ]:
# Difficult...



<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

# Exercise 5.2: Triangles in a graph

A triangle in a graph is a set of 3 vertices inter-connected by 3 edges. Researching and analysing triangles in a graph is essential in understanding its structure.


**a)**
Determine the number of 3-element subsets of vertices in a graph.

In [6]:
from math import comb

def subset_3(g):
    return comb(len(g), 3)
        

subset_3(g)

84

**b)**
Write a function returning the set of all triangles of a graph.

In [7]:
def triangle(g):
    all_edges = edges(g)
    triangles = set()
    
    # First we loop through all edges of the graph, which are pairs of 2 neighboring vertices. If these vertices share
    # some common neighbors, then these 3 vertices form a triangle.
    for v1, v2 in all_edges:
        intersect = g[v1].intersection(g[v2])
        for v3 in intersect:
            triangles.add(tuple(sorted([v1, v2, v3])))
    return triangles

triangle(g)

{(1, 2, 5), (7, 8, 9)}

Each graph $G=(X,E)$ corresponds to a unique symmetrical matrix $A$ of size $n \times n$ with $n=|X|$ defined by:

$$
A_{ij}=\begin{cases}
1\,, & \text{si}\;\{i,j\}\in E\,,\\ 
0\,, & \text{si}\;\{i,j\}\notin E\,.
\end{cases}
$$

This matrix is called the matrix of adjacency of the graph $G$.

**c)**
Define a function returning the matrix of adjacency of a graph.

In [8]:
import numpy as np

def adjacency_matrix(g):
    # Assuming all vertices are numbered correctly starting from 1
    # First, grab all edges of the graph, then create a square zero matrix of size len(g) (number of vertices)
    all_edges = edges(g)
    matrix = np.zeros((len(g),len(g)), dtype='int8')
    
    # Loop through all edges to grab their 2 vertices, then modify directly the corresponding matrix cells
    # [i-1,j-1] and [j-1,i-1]
    for (i, j) in all_edges:
        matrix[i-1,j-1] = 1
        matrix[j-1,i-1] = 1
    return matrix

adjacency_matrix(g)

array([[0, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 1, 0, 0, 0],
       [1, 1, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 0, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 0]], dtype=int8)


**d)**
Define a function taking a matrix of adjacency as argument and returning a dictionary representation of the graph.

In [9]:
def matrix_to_dict(m):
    graph = dict()
    
    # Create empty sets as values of keys representing vertices
    for n in range(len(m)):
        graph[n+1] = set()
    
    # Loop through the upper triangle of the matrix, since the lower contains the same information 
    for i in range(len(m)-1):
        for j in range(i+1,len(m)):
            if m[i][j]:
                graph[i+1].add(j+1)
                graph[j+1].add(i+1)
    return graph

m = adjacency_matrix(g)

matrix_to_dict(m)

{1: {2, 5},
 2: {1, 3, 5},
 3: {2, 4},
 4: {3, 5, 6},
 5: {1, 2, 4},
 6: {4},
 7: {8, 9},
 8: {7, 9},
 9: {7, 8}}


**e)**
Using the matrix of adjacency $A$ and the matrix $B=A^2$, write a function returning the set of all triangles of a graph.

In [10]:
def set_of_triangles(A):
    # The algorithm of this function is to calculate the square of the adjacency matrix A where the value of cell
    # [i, j] = n is the number of distinct paths between vertices i and j of length 2, meaning there is/are
    # exactly n unique paths (i, X, j) in which X is a common neighbor of i and j.
    
    # Thus, if there is at least one path from i to j, the only thing left is to verify if i and j are connected
    # themselves, by looking up their corresponding cell in the original adjacency matrix. If so, we can look for
    # their set of common neighbors {k}, and conclude that {i, k, j} forms a triangle.
    
    B = np.linalg.matrix_power(A,2)
    n = len(A)
    triangles = set()
    
    for i in range(n-1):
        for j in range(i+1,n):
            if B[i,j] and A[i,j]:
                for k in range(n):
                    if A[i,k] == A[j,k] == 1:
                        triangles.add(tuple(sorted([i+1, j+1, k+1])))
    return triangles

set_of_triangles(m)    

{(1, 2, 5), (7, 8, 9)}


**f)**
Using the matrix of adjacency $A$, write a function calculating the number of triangles in a graph.

<!-- --- begin hint in exercise --- -->

**Indication:**
Interpret the meaning of matrix $A^n$.

<!-- --- end hint in exercise --- -->



In [11]:
def num_of_triangles(A):
    # First, it is important to understand that for all positive integer non-null n, for each cell in the matrix A^n,
    # its value represents the number of distinct paths of length n between the 2 vertices (row, col) in the graph
    # whose matrix of adjancency is A.
    
    # This function uses the fact that the number of triangles of a graph is one sixth of the trace of the cubic
    # of the adjacency matrix. Proof: if there exists a triangle (i, j, k) then there are 2 cycles of length 3
    # associated with this triangle starting and ending with each vertex i, j and k. Indeed, with vertex i,
    # there are 2 cycles [i, j, k, i] and [i, k, j, i]. So for one triangle there are 6 cycles. The diagonal of the
    # cubic matrix of A represents the number of 3-cycles starting and ending in one vertex (a triangle), hence
    # one sixth of the trace.

    B = np.linalg.matrix_power(A,3)
    return np.trace(B)//6  # Use floor division to return an integer value

num_of_triangles(m)

2